#   MTCNN Face Detection - Implantação

*   Este componente utiliza a biblioteca [facenet-pytorch](https://github.com/timesler/facenet-pytorch), a qual disponibiliza o algorítimo [MTCNN](https://arxiv.org/abs/1604.02878). 

* O MTCNN possui a performance estado da arte nos benchmarks [FDDB](http://vis-www.cs.umass.edu/fddb/) e [WIDER FACE](http://shuoyang1213.me/WIDERFACE/)

*   Melhores explicações são encontradas neste [artigo do kaggle](https://www.kaggle.com/timesler/guide-to-mtcnn-in-facenet-pytorch)


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [35]:
!pip install facenet-pytorch --quiet

In [36]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
import base64
import cv2
from mtcnn import MTCNN_Model

        
class Model(object):
    
    def __init__(self):
        artifacts = joblib.load("/tmp/data/mtcnn.joblib")
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
    def class_names(self):
        
        return ['bboxcoord1', 'bboxcoord2', 'bboxcoord3', 'bboxcoord4', 'probability']
        
    def format_result(self, bboxes, probs):
        
        res = []
        
        for bbox_id, prob in enumerate(probs):
            
            # Check if has found a bbox for the image
            if prob is None:
                bbox = [None, None, None, None, None]
            
            # Extend to an array the 4coords and prob
            else:
                bbox = list(map(float, bboxes[bbox_id]))
                bbox.extend([float(probs[bbox_id])])
                
            res.append(bbox)
        
        return np.array(res)
    
    def predict(self, X, feature_names, meta=None):
        
        # Check if data is a bytes
        if isinstance(X, bytes):
            im_bytes = X # Get image bytes
        
        # If not, should be a list or ndarray
        else:
            # Garantee is a ndarray
            X = np.array(X)
            
            # Seek for extra dimension
            if len(X.shape) == 2:
                im_bytes = X[0,0] # Get image bytes
            
            else:
                im_bytes = X[0] # Get image bytes
        
        # Preprocess img bytes to img_arr
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img_arr = np.array(img)
        
        # Initializate model
        model = MTCNN_Model(self.model_parameters, self.inference_parameters)
        
        # Predict results
        bboxes, probs = model.predict(img_arr)
        
        # Remove batch dimension
        bboxes, probs = np.squeeze(bboxes, 0), np.squeeze(probs, 0)
        
#         ### DEBUG ###
#         print('--- DBG ---')
#         print('bboxes:', bboxes)
#         print('--- DBG ---')
#         print('probs:', probs)
#         print('--- DBG ---')
#         #############
        
        # Compile results
        result = self.format_result(bboxes, probs)
        
#         ### DEBUG ###
#         print('--- DBG ---')
#         print('result:', result)
#         print('--- DBG ---')
#         #############

        return result

Overwriting Model.py
